In [1]:
import os
import glob
import optuna
import warnings
import h5py as h5
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from copy import deepcopy

from gensit.config import Config
from gensit.inputs import Inputs
from gensit.outputs import Outputs
from gensit.utils.misc_utils import *
from gensit.utils.math_utils import *
from gensit.utils.probability_utils import *
from gensit.contingency_table import instantiate_ct
from gensit.contingency_table.MarkovBasis import instantiate_markov_basis

In [27]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import samples

In [28]:
# Specify experiment id
experiment_id = "JointTableSIM_NN_SweepedNoise_01_02_2024_14_42_49"
# Specify experiment group id
experiment_group_id = 'exp2/'
experiment_dir = f'../data/outputs/cambridge_work_commuter_lsoas_to_msoas/{experiment_group_id}/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [29]:
# Create new logging object
logger = setup_logger(
    __name__,
    console_level = 'INFO',
    file_level = 'EMPTY'
)

In [30]:
# Output processing settings
settings = {
    "logging_mode": "INFO",
    "coordinate_slice": [
        "da.loss_name.isin([str(['dest_attraction_ts_likelihood_loss']),str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])])",
        "~da.title.isin(['_unconstrained','_total_intensity_row_table_constrained'])",
        # "da.loss_name == str(['dest_attraction_ts_likelihood_loss'])",
        # "~da.title.isin([str('_unconstrained'), str('_total_constrained')])"
    ],
    "slice":True,
    "metadata_keys":[],
    "burnin_thinning_trimming": [], # {'iter': {"burnin":10000, "thinning":90, "trimming":1000}}
    "sample":["table"],
    "group_by":['seed'],
    "filename_ending":"test",
    "force_reload":False,
    "n_workers": 1
}

In [31]:
# Initialise outputs
current_sweep_outputs = Outputs(
    config = os.path.join(relative_experiment_dir,"config.json"),
    settings = settings,
    inputs = None,
    slice = True,
    level = 'NOTE'
)
# Silence outputs
current_sweep_outputs.logger.setLevels(console_level='NOTE')
# Load all data
current_sweep_outputs.load()

24:57.206 config INFO ----------------------------------------------------------------------------------
24:57.220 config INFO Parameter space size: 
 --- title: ['title', 'axes', 'cells', 'name', 'proposal'] (7)
 --- seed: ['seed', 'N'] (1333)
 --- sigma: ['sigma', 'to_learn'] (3)
 --- loss_name: ['loss_name', 'loss_function', 'loss_kwargs'] (2)
24:57.234 config INFO Total = 55986.
24:57.248 config INFO ----------------------------------------------------------------------------------
Reading table group data:   0%|          | 1/294 [00:00<00:56,  5.21it/s]

here True
here True


Reading table group data:   1%|          | 3/294 [00:00<00:41,  7.02it/s]

here True
here True


Reading table group data:   2%|▏         | 5/294 [00:00<00:41,  7.01it/s]

...

Reading table group data:  99%|█████████▉| 291/294 [00:39<00:00,  6.39it/s]

here True
here True


Reading table group data: 100%|█████████▉| 293/294 [00:39<00:00,  6.59it/s]

here True
here True


Grouping/Initialising Data Collection samples sequentially: 100%|██████████| 105/105 [00:00<00:00, 106402.98it/s]
Combining Data Collection group elements: 100%|██████████| 105/105 [00:00<00:00, 1117771.37it/s]


In [32]:
# Get data from first sweep of the SIM_NN experiment
current_sweep_outputs0 = current_sweep_outputs.get(0)

In [33]:
# Get table samples
current_sweep_outputs0.data.table

<xarray.DataArray (seed: 1, iter: 10000, origin: 69, destination: 13, sweep: 1)>
array([[[[[172.],
          [  1.],
          [273.],
          ...,
          [  1.],
          [ 60.],
          [  1.]],

         [[  1.],
          [  1.],
          [  1.],
          ...,
          [  1.],
          [ 41.],
          [351.]],

         [[  1.],
          [  1.],
          [  1.],
          ...,
...
          ...,
          [ 15.],
          [ 76.],
          [ 72.]],

         [[  5.],
          [  3.],
          [ 31.],
          ...,
          [ 11.],
          [ 43.],
          [ 52.]],

         [[  7.],
          [  4.],
          [ 41.],
          ...,
          [  5.],
          [ 43.],
          [ 35.]]]]], dtype=float32)
Coordinates: (12/16)
  * seed           (seed) int32 1333
  * iter           (iter) int32 1 2 3 4 5 6 7 ... 9995 9996 9997 9998 9999 10000
  * origin         (origin) int16 1 2 3 4 5 6 7 8 9 ... 62 63 64 65 66 67 68 69
  * destination    (destination) int16 1 2 3 4 5 6 7 8 9 10 11 12 13
  * sweep          (sweep) object MultiIndex
  * sigma          (sweep) <U4 'none'
    ...             ...
  * name           (sweep) <U18 'TotallyConstrained'
  * proposal       (sweep) <U13 'degree_higher'
  * N              (sweep) int32 10000
  * loss_name      (sweep) <U63 "['dest_attraction_ts_likelihood_loss', 'tabl...
  * loss_function  (sweep) <U20 "['custom', 'custom']"
  * loss_kwargs    (sweep) <U26 "{'noise_percentage': None}"
Attributes:
    arr_name:       table
    experiment_id:  JointTableSIM_NN_LearnedNoise__doubly_20%_cell_constraine...
    sweep_id:       loss_name_[dest_attraction_ts_likelihood_loss,table_likel...